### Solution

Pivot columns 제거   
drop_feature = ['시간당분무량', '시간당백색광량', '시간당적색광량', '시간당청색광량', '시간당총광량', 'index']  
6시간 이동평균  
catboost  


In [23]:
from feature.make_dataset import make_data
from feature.after_dataset import accumulate
import pandas as pd
from glob import glob
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import glob
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')


In [24]:
train_input_list = './data/train_input/*.csv'
train_target_list = './data/train_target/*.csv'

test_input_list = './data/test_input/*.csv'
test_target_list = './data/test_target/*.csv'

In [25]:
train, test, p_train, p_test = make_data(train_input_list, train_target_list, test_input_list, test_target_list)
train_rol = pd.read_csv("train_roling.csv")
test_rol = pd.read_csv("test_roling.csv")

In [26]:
train_rol = train_rol.drop(['DAT'], axis=1)
test_rol = test_rol.drop(['DAT'], axis=1)

In [27]:
train = train.reset_index()
test = test.reset_index()

In [28]:
accumulate(train, test, '누적분무량')
accumulate(train, test, '누적백색광량')
accumulate(train, test, '누적청색광량')
accumulate(train, test, '누적적색광량')
accumulate(train, test, '누적총광량')

In [29]:
train = pd.concat([train, train_rol], axis=1)
test = pd.concat([test, test_rol], axis=1)

In [30]:
drop_feature = ['시간당분무량', '시간당백색광량', '시간당적색광량', '시간당청색광량', '시간당총광량', 'index']
train = train.drop(drop_feature, axis=1)
test = test.drop(drop_feature, axis=1)


In [31]:
def model_train(train, test, model):
    '''
    train shape = (784, 18), test shape = (140, 18)
    model => sklearn api model
    '''

    X = train.drop(['predicted_weight_g', 'Case', 'obs_time'], axis=1)
    y = train['predicted_weight_g']

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=113, shuffle=True)
    
    x_test = test.drop(['predicted_weight_g', 'Case', 'obs_time'], axis=1)
    
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)

    rmse = mean_squared_error(y_val, y_val_pred)**0.5
    print(f"validation rmse: {rmse}")
    
    x_train = train.drop(['predicted_weight_g', 'Case', 'obs_time'], axis=1)
    y_train = train['predicted_weight_g']
    
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    x_test['predicted_weight_g'] = y_pred
    submit = x_test[['DAT', 'predicted_weight_g']]
    submit['DAT'] = submit['DAT']+1
    all_target_list = sorted(glob.glob('./data/test_target/*.csv'))
    for idx, test_path in enumerate(all_target_list):
        submit_df = pd.read_csv(test_path)
        submit_df['predicted_weight_g'] = submit['predicted_weight_g'][idx*28:idx*28+28].values
        submit_df.to_csv(test_path, index=False)

In [39]:
xgb = XGBRegressor()
cat = CatBoostRegressor(verbose=100, early_stopping_rounds=50, task_type="GPU", random_seed=113)
lgbm = LGBMRegressor()
linear = LinearRegression()

model_train(train, test, cat)

Learning rate set to 0.037333
0:	learn: 40.7770135	total: 10.4ms	remaining: 10.4s
100:	learn: 7.3718252	total: 1s	remaining: 8.95s
200:	learn: 5.5672474	total: 1.99s	remaining: 7.92s
300:	learn: 5.1582953	total: 2.96s	remaining: 6.87s
400:	learn: 4.9197798	total: 3.93s	remaining: 5.87s
500:	learn: 4.6914787	total: 4.9s	remaining: 4.88s
600:	learn: 4.3552627	total: 5.86s	remaining: 3.89s
700:	learn: 4.2618948	total: 6.84s	remaining: 2.92s
800:	learn: 4.1930470	total: 7.8s	remaining: 1.94s
900:	learn: 4.0190300	total: 8.78s	remaining: 965ms
999:	learn: 3.8514780	total: 9.76s	remaining: 0us
validation rmse: 8.452685654848711
Learning rate set to 0.039126
0:	learn: 40.3119690	total: 10.5ms	remaining: 10.5s
100:	learn: 6.8254443	total: 1.02s	remaining: 9.08s
200:	learn: 5.1202895	total: 2.05s	remaining: 8.13s
300:	learn: 4.7459278	total: 3.09s	remaining: 7.17s
400:	learn: 4.3773633	total: 4.14s	remaining: 6.18s
500:	learn: 4.2403114	total: 5.18s	remaining: 5.16s
600:	learn: 4.0539905	total:

In [40]:
df_imp = pd.DataFrame({'imp':cat.feature_importances_}, index = cat.feature_names_)
df_imp = df_imp[df_imp.imp > 0].sort_values('imp').copy()
df_imp

,imp
"('일간누적분무량', 11)",0.000020
"('ec관측치', 17)",0.000032
"('ec관측치', 4)",0.000048
"('co2관측치', 13)",0.000058
"('일간누적분무량', 23)",0.000091
...,...
"('내부온도관측치', 14)",3.737963
월간누적적색광량,8.319814
월간누적백색광량,12.471388
DAT,20.066507


In [36]:
train

,DAT,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,일간누적분무량,일간누적백색광량,일간누적적색광량,일간누적청색광량,...,"('일간누적총광량', 14)","('일간누적총광량', 15)","('일간누적총광량', 16)","('일간누적총광량', 17)","('일간누적총광량', 18)","('일간누적총광량', 19)","('일간누적총광량', 20)","('일간누적총광량', 21)","('일간누적총광량', 22)","('일간누적총광량', 23)"
0,0,11.5,27.030709,78.061290,535.094296,1.393616,3961.27,255677.8594,13698.4344,0.0000,...,124369.525000,143601.0470,162832.5690,182064.091000,201295.613000,220527.135000,236681.795633,249631.202600,259375.355900,265914.255533
1,1,11.5,27.542751,78.075041,523.986590,1.384431,3980.56,256324.5263,13708.3632,0.0000,...,124678.935000,143910.4570,163141.9790,182373.501000,201605.023000,220836.545000,237049.069917,250056.341167,259858.358750,266455.122667
2,2,11.5,26.678482,77.590048,539.236170,1.392745,4093.70,255724.2709,13675.2672,0.0000,...,124239.575433,143416.7711,162648.2931,181879.815100,201111.337100,220342.859100,236532.106433,249516.100100,259294.840100,265868.326433
3,3,11.5,26.157335,77.366091,760.150330,1.401298,3980.56,256324.5263,13708.3632,0.0000,...,124678.935000,143910.4570,163141.9790,182373.501000,201605.023000,220836.545000,237049.069917,250056.341167,259858.358750,266455.122667
4,4,11.5,26.522891,78.131650,539.535758,1.403796,3935.55,255829.4703,13681.8864,0.0000,...,124297.067700,143528.5897,162760.1117,181991.633700,201223.155700,220454.677700,236643.925033,249627.918700,259406.658700,265980.145033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,23,11.5,27.901224,58.783863,457.965007,1.156525,27895.87,84543.1884,12856.1412,6038.8575,...,42712.123333,50584.1140,58677.2280,68052.857333,77523.692467,85669.334650,92532.461500,98068.848350,102057.371867,103436.640050
780,24,11.5,25.261621,59.967900,451.424429,1.189204,3292.69,146626.3049,22339.8000,10503.3825,...,85288.697000,100944.2290,116599.7610,132255.293000,146646.432367,158429.347767,167603.007833,174167.412567,178122.561967,179468.456033
781,25,11.5,24.919758,60.240364,459.397150,1.191183,2995.68,146617.0226,22319.9424,10495.5500,...,85055.259400,100710.7914,116366.3234,132021.855400,146445.223283,158260.882883,167467.287150,174064.436083,178052.329683,179430.967950
782,26,11.5,25.478958,60.134861,428.962500,1.171493,2620.28,146632.4931,22266.9888,10473.6190,...,85078.171500,100733.7035,116389.2355,132044.767500,146453.898667,158255.837233,167448.520467,174031.948367,178006.120933,179371.038167


In [37]:
train.corr()

,DAT,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,일간누적분무량,일간누적백색광량,일간누적적색광량,일간누적청색광량,...,"('일간누적총광량', 14)","('일간누적총광량', 15)","('일간누적총광량', 16)","('일간누적총광량', 17)","('일간누적총광량', 18)","('일간누적총광량', 19)","('일간누적총광량', 20)","('일간누적총광량', 21)","('일간누적총광량', 22)","('일간누적총광량', 23)"
DAT,1.000000,NaN,-0.102163,0.503426,-0.191158,0.021341,0.091901,-0.077505,-0.033900,0.149023,...,0.010883,-0.002001,-0.013631,-0.022635,-0.028553,-0.031664,-0.032913,-0.033973,-0.034178,-0.033722
obs_time,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
내부온도관측치,-0.102163,NaN,1.000000,-0.041613,-0.046338,-0.187750,-0.057464,-0.078630,-0.019511,-0.126554,...,-0.092421,-0.108594,-0.123352,-0.134517,-0.142698,-0.148571,-0.152401,-0.156697,-0.160483,-0.163517
내부습도관측치,0.503426,NaN,-0.041613,1.000000,-0.162003,0.141425,0.011774,-0.003454,0.046956,0.164026,...,0.214706,0.189568,0.165471,0.144041,0.128968,0.122558,0.117454,0.114254,0.112905,0.114063
co2관측치,-0.191158,NaN,-0.046338,-0.162003,1.000000,-0.090468,-0.463899,0.290799,-0.234993,-0.131105,...,0.046266,0.099629,0.144051,0.178434,0.201857,0.213530,0.215418,0.209911,0.198335,0.181321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"('일간누적총광량', 19)",-0.031664,NaN,-0.148571,0.122558,0.213530,-0.179940,-0.329041,0.719513,-0.012899,0.165840,...,0.913459,0.958151,0.983287,0.994918,0.999075,1.000000,0.999208,0.997008,0.992768,0.985823
"('일간누적총광량', 20)",-0.032913,NaN,-0.152401,0.117454,0.215418,-0.173800,-0.325578,0.714981,-0.001580,0.173192,...,0.909460,0.954193,0.979692,0.991853,0.996916,0.999208,1.000000,0.999174,0.996275,0.990611
"('일간누적총광량', 21)",-0.033973,NaN,-0.156697,0.114254,0.209911,-0.165690,-0.318432,0.703433,0.019540,0.189177,...,0.911424,0.953962,0.977905,0.989069,0.993951,0.997008,0.999174,1.000000,0.998913,0.995136
"('일간누적총광량', 22)",-0.034178,NaN,-0.160483,0.112905,0.198335,-0.154561,-0.307975,0.684135,0.049483,0.212196,...,0.916477,0.955253,0.976277,0.985354,0.989300,0.992768,0.996275,0.998913,1.000000,0.998620
